In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

from keras.metrics import mean_absolute_error
from keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

VOCAB_SIZE = 15000
SEQUENCE_LENGTH = 100
EMBED_DIM = 8

DATA_PATH=r"../../data/transformed/amazon_reviews_5_partition_1.csv"

In [ ]:
df_reader = pd.read_csv(
    DATA_PATH,
    index_col=0,
    chunksize=10000
)

In [ ]:
df = df_reader.__next__()

In [ ]:
# df = pd.read_csv(DATA_PATH, index_col=0)

In [ ]:
df.sample(4)

In [ ]:
df = df.dropna()
df["reviewText"] = df["reviewText"].astype("string")

In [ ]:
with open("../../checkpoints/vectorization_vocabulary.txt", "r") as file:
    vocab = file.read()
    vocab=vocab.split("\n")[:-1] #last line is an empty string

vectorize_layer = TextVectorization(
    VOCAB_SIZE,
    "lower_and_strip_punctuation",
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    vocabulary=vocab
)

In [ ]:
def to_vec_array(x):
    return np.c_[x]

In [ ]:
x = df["reviewText"].apply(vectorize_layer)
y = df["overall"] -1 # Now [0-4] scoring

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
model = SVR()

In [ ]:
model.fit(x.to_list(), y)

In [ ]:
mean_absolute_error(y_test, model.predict(x_test.to_list())).numpy()

In [ ]:
def predict_svr(text):
    x = np.array(vectorize_layer(text)).reshape(1, -1)
    return model.predict(x)

In [ ]:
predict_svr("dead dead dead dead dead dead")

In [ ]:
df_testing = df.iloc[:20,:2]

In [ ]:
df_testing["preds"] = df_testing["reviewText"].apply(predict_svr)

In [ ]:
df_testing